In [4]:

from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(1, 1).to(device)

batch_size = 64

epochs = 40
criterion = nn.MSELoss()


optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-2},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-2},
        {"params": modello.final.parameters()}
    ],
    lr=1e-1,
)
early_stop = sp.EarlyStopping(7, 0.001)

data, references, descriptions, target = get_tabular(aggregated_path, desc_path)

train, val = getDataset(references, data, descriptions, target, img_path, batch_size, 1)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
min_mae = 100000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    if (min_mae > mae):
        modello_final = modello
        min_mae = mae
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.')
        break
    
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\season_nonegozi')
model_sc = torch.jit.script(modello_final)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\season_nonegozi\\weights.pt')

Sto usando cuda


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\preproc_tabular.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted b

Epoch 1
-------------------------------
MSE: 5879469.000000  [   64/ 1111]
In questa epoca, l'MSE medio è 5061592.0
Validation Error: 
 Avg MSE: 3414901.531250 
 Avg MAE: 1639.234253 
 Avg R2: -2.682207
 Bias: -1638.453769

Epoch 2
-------------------------------
MSE: 4965682.000000  [   64/ 1111]
In questa epoca, l'MSE medio è 3378186.75
Validation Error: 
 Avg MSE: 1121372.078125 
 Avg MAE: 877.144958 
 Avg R2: -0.214950
 Bias: -725.112249

Epoch 3
-------------------------------
MSE: 2158082.500000  [   64/ 1111]
In questa epoca, l'MSE medio è 1266575.75
Validation Error: 
 Avg MSE: 2939788.875000 
 Avg MAE: 1516.053711 
 Avg R2: -2.166046
 Bias: 1374.113539

Epoch 4
-------------------------------
MSE: 753472.375000  [   64/ 1111]
In questa epoca, l'MSE medio è 644618.8125
Validation Error: 
 Avg MSE: 1752448.906250 
 Avg MAE: 1049.850479 
 Avg R2: -0.880796
 Bias: 937.052734

Epoch 5
-------------------------------
MSE: 489758.343750  [   64/ 1111]
In questa epoca, l'MSE medio è 4

FileExistsError: [WinError 183] Impossibile creare un file, se il file esiste già: 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\season_nonegozi'